# Lesson S3 - Regridding NLDAS forcing data to the WRF-Hydro domain

## Overview
In this lesson, we cover regridding NLDAS2 forcing data to a WRF-Hydro domain using the Python-based WRF-Hydro Forcing Engine

## Required software and datasets

### Meteorological Forcing Engine (MFE) software
The WRF-Hydro Meteorological Forcing Engine (MFE) software can be downloaded from https://github.com/NCAR/WrfHydroForcing

### Data sources
For this training, data has already been included for the simulation time period. 

**NOTE: These data are for instructional purposes only and do not include forcing data used for model spinup**

If you would like to acquire other data, NLDAS2 forcing data can be retrieved from the NASA data server at https://hydro1.gesdisc.eosdis.nasa.gov/data/NLDAS/NLDAS_FORA0125_H.002/. 

See the following site for information on how to access data:
https://disc.gsfc.nasa.gov/data-access 

See the following instructions for how to download data files using wget:
https://disc.gsfc.nasa.gov/information/howto?title=How%20to%20Download%20Data%20Files%20from%20HTTP%20Service%20with%20wget

**Step 1: Extract the NLDAS2 forcing data provided with this tutorial**

In [ ]:
%%bash
cd ~/wrf-hydro-training/regridding
tar -xf nldas_mfe_forcing.tar.gz

**Step 2: View the contents of the input_files directory**

In [ ]:
%%bash
ls ~/wrf-hydro-training/regridding/NLDAS/input_files | head -10

Let's view the data in the NLDAS2 inputs:

In [ ]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/regridding/NLDAS/input_files/NLDAS_FORA0125_H.A20170101.*.002.nc')

input_field = ldas_in.APCP_surface

input_field.hvplot(groupby='time', data_aspect=1.0, xaxis=None, yaxis=None,
                   title=input_field.short_name, attr_labels=False, widget_location='bottom',
                   cmap='gist_ncar', clim=(input_field.min(), input_field.max()))

**Step 3: View the domain metadata files**

The Forcing Engine requires two additional static input files; a WRF *geogrid* file (for grid information), and an optional Spatial Metadata template file to copy attributes from.

In [ ]:
%%bash
ls ~/wrf-hydro-training/regridding/NLDAS/*nc

We have the necessary files, now we can begin the regridding process.

## Editing the forcing engine configuration file
The WRF-Hydro Forcing Engine requires a configuration file to specify the input types, forecast time periods, and other options needed for regridding and pre-processing the forcing data.

**Step 1: Examine the supplied *nldas_fe.config* configuration file and verify the times and forcing input types**

In [ ]:
%%bash
cat ~/wrf-hydro-training/regridding/NLDAS/nldas_fe.config

## Regridding the forcing data using the forcing engine
In this section we will regrid all our forcing data using the configuration shown above

**Step 1: Run the Forcing Engine driver (`genForcing.py`)**

`genForcing.py` takes 3 arguments: the path to the configuration file, a version number, and a configuration name. The version and name are arbitrary and used for logging and are inserted as netCDF Global Attributes in the LDASIN output files.

In [ ]:
%%bash
cd ~/wrf-hydro-training/regridding/NLDAS
rm -r output_files/* LOG_NLDAS*          # clean up any existing runs
python3 ../../WrfHydroForcing/genForcing.py nldas_fe.config 5.2.1 NLDAS

The Forcing Engine sends most of its output to a log file specified in the configuration file. Only severe errors are printed to *stdout* so a successful run will produce no output.

**Step 2: View the Forcing Engine Log:**

In [ ]:
%%bash
cat /home/docker/wrf-hydro-training/regridding/NLDAS/LOG_NLDAS*

**Step 3: View the contents of the *output_files* directory**

The Forcing Engine will generate output files in a directory heirarchy rooted in `output_files`. Lets view the contents of the directory:

In [ ]:
%%bash
ls -R ~/wrf-hydro-training/regridding/NLDAS/output_files

**Step 4: View the contents of the LDASIN files**

Now, let's look at the actual regridded data in the output files. Try changing the variable from "RAINRATE" to others in the file to see the other fields.

In [ ]:
import xarray as xr
import hvplot.xarray

ldas_in = xr.open_mfdataset('/home/docker/wrf-hydro-training/regridding/NLDAS/output_files/2017010100/2017010*.LDASIN_DOMAIN1')

ldas_in

In [ ]:
ldas_in.RAINRATE.hvplot(groupby='time', data_aspect=1.0, xaxis=None, yaxis=None,
                        title=input_field.name, attr_labels=False, widget_location='bottom',
                        cmap='gist_ncar', clim=(input_field.min(), input_field.max()))

© UCAR 2020